In [320]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
from scipy.stats import kurtosis
from numpy import array
from matplotlib import cm
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import Imputer
from sklearn.externals import joblib
import datetime
import time
# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train = joblib.load('train1.pkl')    # 载入数据

In [3]:
listing_info = joblib.load('listing_info1.pkl')    # 载入数据

In [4]:
user_repay_logs = joblib.load('user_repay_logs1.pkl')    # 载入数据

In [ ]:
user_behavior_logs = joblib.load('user_behavior_logs1.pkl')    # 载入数据

In [5]:
user_info = joblib.load('user_info1.pkl')    # 载入数据

In [6]:
user_taglist = joblib.load('user_taglist1.pkl')    # 载入数据

In [7]:
test = joblib.load('test1.pkl')    # 载入数据

In [8]:
# 将test中有历史交易记录的user对应记录提取出来作为train1
train_user_id=set(train['user_id'].tolist())
test_user_id=set(test['user_id'].tolist())
repay_logs_user_id = set(user_repay_logs['user_id'].tolist())
repay_logs_listing_id = set(user_repay_logs['listing_id'].tolist())
train_listing_id=set(train['listing_id'].tolist())

In [9]:
x1 = test_user_id - train_user_id
user_no = x1 - repay_logs_user_id    # test中没在train、repay_log中出现过的user
user_no_num = test[test['user_id'].isin(user_no)]       # 保存test中从未出现在train、repay_log中的部分
user_yes_num = test[~test['user_id'].isin(user_no)]     # 保存test中在train、repay_log中出现过的部

In [10]:
user_yes_train = train[train['user_id'].isin(test_user_id&train_user_id)]   
# test中在train出现过的user

print(user_yes_train.shape)

(33379, 7)


In [20]:
user_yes_train.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
52,399026,5105859,2018-12-12,2019-01-12,396.0697,2019-01-12,396.0697
129,491230,4773399,2018-10-28,2018-11-28,943.9129,2018-11-23,943.9129
148,889013,4916211,2018-11-15,2018-12-15,170.1841,2018-12-15,170.1841
153,493599,2393635,2018-02-23,2018-03-23,390.6251,2018-03-22,390.6251
157,741140,4460746,2018-09-17,2018-10-17,394.6893,2018-10-16,394.6893


In [11]:
user_yes_log = user_repay_logs[user_repay_logs['user_id'].isin(test_user_id&repay_logs_user_id)]   
# test中在log出现过的user

print(user_yes_log.shape)

(2746766, 7)


In [12]:
def logpart(x): 
    # 处理日志数据
    x['due_date'] = pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['repay_date'] = pd.to_datetime(x.repay_date,format = '%Y-%m-%d')
    x['month'] = x['due_date'].map(lambda x:x.month)
    x['year'] = x['due_date'].map(lambda x:x.year)
    x['day'] = x['due_date'].map(lambda x:x.day)
    x['delta'] = x['month'] - x['order_id']
    x.loc[x['delta'] <= 0,'month'] = x.loc[x['delta'] <= 0,'delta'].map(lambda x: x+12) 
    x.loc[x['delta'] <= 0,'year'] = x.loc[x['delta'] <= 0,'year'].map(lambda x: x-1)
    x.loc[x['delta'] > 0,'month']= x.loc[x['delta'] > 0,'delta']
    x.loc[x['delta'] > 0,'year'] = x.loc[x['delta'] > 0,'year']
    x['year'] =  [int(i) for i in x['year']]
    x['year'] = x['year'].astype(str)
    x['month'] =  [int(i) for i in x['month']]
    x['month'] = x['month'].astype(str)
    x['day'] = x['day'].astype(str)
    x['auditing_date'] = x['year'].str.cat([x['month'],x['day']],sep='-')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    x.drop(['order_id','month','year','day','delta'],axis=1,inplace=True)
    order = ['user_id','listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt']
    x = x[order]
    return x     # 日期数据都转成了日期格式

In [13]:
def trainpart(x):
    # 处理train数据
    x.loc[x['repay_date'] == '\\N','repay_date']= '2200-01-01'
    x.loc[x['repay_amt'] == '\\N','repay_amt'] = x.loc[x['repay_amt'] == '\\N','due_amt']
    x['due_date'] = pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['repay_date'] = pd.to_datetime(x.repay_date,format = '%Y-%m-%d')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    return x

In [14]:
def testpart(x):
    # 处理测试集数据
    x['due_date']= pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    return x

In [15]:
user_yes_train1 = trainpart(user_yes_train)
user_yes_log1 = logpart(user_yes_log)
test1 = testpart(test)

In [16]:
user_yes_log1.head()     # 无缺失值

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
59,381204,3380189,2018-05-13,2018-10-13,287.0677,2018-09-16,287.0677
60,381204,3380189,2018-05-13,2018-11-13,287.0677,2018-09-25,287.0677
61,381204,3380189,2018-05-13,2018-09-13,287.0677,2018-08-15,287.0677
84,657194,3762755,2018-06-17,2018-07-17,121.3392,2018-07-17,121.3392
85,657194,3762755,2018-06-17,2018-11-17,121.3392,2018-11-17,121.3392


In [18]:
user_yes_train1.isnull().sum()      # 无缺失值

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
repay_date       0
repay_amt        0
dtype: int64

In [19]:
user_yes_log2 = user_yes_log1.sort_values(by=["repay_date"]).set_index(["repay_date"])

In [26]:
user_yes_log2.head()
print(user_yes_log2.shape)

(2746766, 6)


In [25]:
log1 = user_yes_log2.loc['2200-01-01'].reset_index()

In [27]:
log1.shape       # 日志数据中逾期的记录量

(132687, 7)

In [28]:
log2 = log1.sort_values(by=["due_date"]).set_index(["due_date"])
log3 = log2.truncate(after="2019-03-01").reset_index()     # 日志中逾期部分数据的筛选

In [29]:
log3.shape

(125689, 7)

In [30]:
user_yes_log1["repay_date"] = user_yes_log1["repay_date"].astype(str)
log4 = user_yes_log1.loc[user_yes_log1["repay_date"]!='2200-01-01']      # 挑出未逾期的记录
log4["repay_date"] = pd.to_datetime(log4.repay_date,format = '%Y-%m-%d')     

In [31]:
log5 = log4.sort_values(by=["repay_date"]).set_index(["repay_date"])
log6 = log5.truncate(after="2019-03-01").reset_index()     # 日志中非逾期部分数据的筛选

In [32]:
log6.shape

(2484832, 7)

In [33]:
order = ['user_id','listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt']
log3 = log3[order]
log6 = log6[order]

In [42]:
log3.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.9782
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.0636
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.2594
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.6113
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.1742


In [43]:
log6.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,270705,6256,2016-09-28,2017-09-28,245.2538,2016-10-01,245.2538
1,270705,6256,2016-09-28,2017-08-28,245.2538,2016-10-01,245.2538
2,270705,6256,2016-09-28,2017-07-28,245.2538,2016-10-01,245.2538
3,282185,10198,2016-10-17,2017-10-17,159.4149,2016-10-17,159.4149
4,282185,10198,2016-10-17,2017-09-17,159.4149,2016-10-17,159.4149


In [44]:
user_yes_train1.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
52,399026,5105859,2018-12-12,2019-01-12,396.0697,2019-01-12,396.0697
129,491230,4773399,2018-10-28,2018-11-28,943.9129,2018-11-23,943.9129
148,889013,4916211,2018-11-15,2018-12-15,170.1841,2018-12-15,170.1841
153,493599,2393635,2018-02-23,2018-03-23,390.6251,2018-03-22,390.6251
157,741140,4460746,2018-09-17,2018-10-17,394.6893,2018-10-16,394.6893


In [48]:
user_yes_train1['repay_amt'] = user_yes_train1['repay_amt'].astype(float)

In [49]:
user_yes_train1.info()     # repay_amt是object类型

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33379 entries, 52 to 999996
Data columns (total 7 columns):
user_id          33379 non-null int64
listing_id       33379 non-null int64
auditing_date    33379 non-null datetime64[ns]
due_date         33379 non-null datetime64[ns]
due_amt          33379 non-null float64
repay_date       33379 non-null datetime64[ns]
repay_amt        33379 non-null float64
dtypes: datetime64[ns](3), float64(2), int64(2)
memory usage: 2.0 MB


In [35]:
log6.isnull().sum()     # 均无缺失值

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
repay_date       0
repay_amt        0
dtype: int64

In [50]:
total = pd.concat([log3,log6,user_yes_train1],axis=0,ignore_index=True)
print(total.shape)

(2643900, 7)


In [51]:
total.drop_duplicates(keep='first',inplace=True)
print(total.shape)     # 删除重复的记录

(2611836, 7)


In [38]:
total.isnull().sum()    # 均无缺失值

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
repay_date       0
repay_amt        0
dtype: int64

In [173]:
len(total['user_id'].unique())

119503

In [53]:
joblib.dump(total, 'total1.pkl')    # 存储合并的训练集

['total1.pkl']

In [7]:
total = joblib.load('total1.pkl')    # 载入数据

In [52]:
total.head()
print(total.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2611836 entries, 0 to 2643883
Data columns (total 7 columns):
user_id          int64
listing_id       int64
auditing_date    datetime64[ns]
due_date         datetime64[ns]
due_amt          float64
repay_date       datetime64[ns]
repay_amt        float64
dtypes: datetime64[ns](3), float64(2), int64(2)
memory usage: 159.4 MB
None


In [54]:
total['days']=(total['due_date']-total['repay_date']).apply(lambda x:x.days)   
# 提前还款天数（标签）

In [55]:
total['days']=total['days'].apply(lambda x: x if x>=0 else -1 )   # 逾期的标为-1

In [58]:
total['days'].describe()

count    2.611836e+06
mean     3.158584e+01
std      6.318459e+01
min     -1.000000e+00
25%      0.000000e+00
50%      2.000000e+00
75%      2.800000e+01
max      3.660000e+02
Name: days, dtype: float64

In [8]:
total.isnull().sum()

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
repay_date       0
repay_amt        0
dtype: int64

In [62]:
sp_total = total[total['days']<32]

In [63]:
sp_total.groupby(by='days').size().sort_values(ascending=False)     # 统计days的频率分布

days
 0     854544
 1     239159
 2     126832
-1     125716
 3     123437
 4      58974
 5      46742
 6      39193
 7      33794
 8      29386
 9      25617
 10     22894
 11     21059
 12     18966
 13     17627
 31     17552
 14     16344
 15     15259
 30     14837
 16     14385
 17     13751
 18     13152
 19     12381
 28     12199
 29     11358
 20     11280
 21     10857
 22     10640
 27     10304
 23     10256
 24     10032
 26      9893
 25      9840
dtype: int64

In [95]:
A = sp_total.groupby(by='user_id').size().sort_values()       # 给出所有交易记录（包括train）中每个user的记录数

In [64]:
sp_total.isnull().sum()   # 无缺失值

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
repay_date       0
repay_amt        0
days             0
dtype: int64

In [70]:
sp_total.shape

(2008260, 8)

In [65]:
total_2 = sp_total.copy()    # 备份筛选后的训练集

In [38]:
sp_total.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt,days
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.98,-1
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.064,-1
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.259,-1
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.611,-1
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.174,-1


In [67]:
total_2[total_2['days']!=-1].shape     # 类别比约为15:1

(1882544, 8)

In [123]:
y_label = sp_total['days'].values

In [69]:
del total_2['repay_amt'],total_2['repay_date'],total_2['days']

In [71]:
testtotal0 = pd.concat([total_2,test1],axis=0,ignore_index=True)
print(testtotal0.shape)                    #  训练集与测试集合并

(2138260, 5)


In [72]:
total_2.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,34612,271495,2017-05-09,2017-07-09,1065.9782
1,34612,8272,2016-10-09,2017-07-09,303.0636
2,153514,20854,2016-11-11,2017-07-11,492.2594
3,260235,403888,2017-06-12,2017-07-12,295.6113
4,155973,69240,2017-01-13,2017-07-13,386.1742


In [73]:
test1.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,498765,5431438,2019-03-12,2019-04-12,138.5903
1,34524,5443211,2019-03-15,2019-04-15,208.0805
2,821741,5461707,2019-03-22,2019-04-22,421.2097
3,263534,5472320,2019-03-26,2019-04-26,212.6537
4,238853,5459750,2019-03-21,2019-04-21,817.4593


In [74]:
testtotal0['year'] = testtotal0['due_date'].dt.year.astype(str)
testtotal0['month'] = testtotal0['due_date'].dt.month.astype(str)
testtotal0['day'] = testtotal0['due_date'].dt.day.astype(str)
testtotal0['dayofweek'] = testtotal0['due_date'].dt.dayofweek.astype(str)

In [75]:
# 应还款日期与交易日间隔天数
testtotal0['days1']=(testtotal0['due_date']-testtotal0['auditing_date']).apply(lambda x:x.days)

In [76]:
testtotal1 = pd.get_dummies(testtotal0)

In [78]:
testtotal2 = pd.merge(testtotal1,listing_info,on=['user_id','listing_id'],how='left')   # 合并标的资产信息

In [81]:
testtotal2.drop(['auditing_date_y'],inplace=True,axis=1)
testtotal2.rename(columns={'auditing_date_x': 'auditing_date'},inplace=True)

In [82]:
testtotal2.isnull().sum()     # 无缺失值

user_id          0
listing_id       0
auditing_date    0
due_date         0
due_amt          0
days1            0
year_2017        0
year_2018        0
year_2019        0
month_1          0
month_10         0
month_11         0
month_12         0
month_2          0
month_3          0
month_4          0
month_5          0
month_6          0
month_7          0
month_8          0
month_9          0
day_1            0
day_10           0
day_11           0
day_12           0
day_13           0
day_14           0
day_15           0
day_16           0
day_17           0
day_18           0
day_19           0
day_2            0
day_20           0
day_21           0
day_22           0
day_23           0
day_24           0
day_25           0
day_26           0
day_27           0
day_28           0
day_29           0
day_3            0
day_30           0
day_31           0
day_4            0
day_5            0
day_6            0
day_7            0
day_8            0
day_9            0
dayofweek_0 

In [83]:
trainnew = testtotal2[:len(total_2)]
testnew = testtotal2[len(total_2):]    # 得到一起处理时间、标的资产特征的训练集、测试集

In [89]:
group = sp_total.groupby('user_id', as_index=False)

In [90]:
total_3 = sp_total.merge(
    group['days'].agg({
        'days_max': 'max', 'days_median': 'median',
        'days_mean': 'mean', 'days_sum':'sum','days_std': 'std', 'days_skew': 'skew', 'days_kurt':kurtosis
    }), on='user_id', how='left'
)    # 还款天数的统计量

In [91]:
total_3.isnull().sum()

user_id              0
listing_id           0
auditing_date        0
due_date             0
due_amt              0
repay_date           0
repay_amt            0
days                 0
days_max             0
days_median          0
days_mean            0
days_sum             0
days_std          8541
days_skew        25765
days_kurt            0
dtype: int64

In [100]:
total_4 = total_3.merge(
    group['due_amt'].agg({
        'due_amt_max': 'max', 'due_amt_min': 'min', 'due_amt_median': 'median',
        'due_amt_mean': 'mean', 'due_amt_sum': 'sum', 'due_amt_std': 'std',
        'due_amt_skew': 'skew', 'due_amt_kurt': kurtosis, 'due_amt_ptp': np.ptp
    }), on='user_id', how='left'
)     # 还款金额的统计量

In [101]:
total_4.isnull().sum()

user_id               0
listing_id            0
auditing_date         0
due_date              0
due_amt               0
repay_date            0
repay_amt             0
days                  0
days_max              0
days_median           0
days_mean             0
days_sum              0
days_std           8541
days_skew         25765
days_kurt             0
due_amt_max           0
due_amt_min           0
due_amt_median        0
due_amt_mean          0
due_amt_sum           0
due_amt_std        8541
due_amt_skew      25765
due_amt_kurt          0
due_amt_ptp           0
dtype: int64

In [276]:
label = total_4['days']

In [261]:
total_5 = total_4.copy()
total_6 = total_4.copy()

In [265]:
total_5.drop(['listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt','days'],inplace=True,axis=1)
total_5.drop_duplicates(keep='first',inplace=True)

In [371]:
test2 = testnew.merge(total_5,on='user_id',how='left')   # 测试集
train2 = trainnew.merge(total_5,on='user_id',how='left')  # 训练集

In [372]:
test3 = test2.drop(['listing_id','auditing_date','due_date','user_id'],axis = 1)   
train3 = train2.drop(['listing_id','auditing_date','due_date','user_id'],axis = 1)  # 没有填补的训练集

In [373]:
test01 = test3[test3['days_max'].isnull()==True]     # 无历史记录的user部分
test02 = test3[test3['days_max'].isnull()==False]     # 有历史记录的user部分

In [ ]:
test01_id = set(test01['user_id'].tolist())        # 对应没有历史交易记录的user_id

In [374]:
test03 = test02.copy()

In [376]:
test02['days_std'] = test02['days_std'].fillna(train3['days_std'].mean())
test02['days_skew'] = test02['days_skew'].fillna(train3['days_skew'].mean())
test02['due_amt_std'] = test02['due_amt_std'].fillna(train3['due_amt_std'].mean())
test02['due_amt_skew'] = test02['due_amt_skew'].fillna(train3['due_amt_skew'].mean())

In [368]:
train4 = train3.fillna(train3.mean())       # 缺失值填补的训练集

In [369]:
y_label     # 标签

array([-1, -1, -1, ...,  3,  0, 31], dtype=int64)

In [370]:
test02.shape                                # 填补了缺失值的测试集

(118125, 74)

In [375]:
joblib.dump(train3, 'train3.pkl')    # 训练集
joblib.dump(test03, 'test03.pkl')    # 训练集

['test03.pkl']

In [377]:
joblib.dump(train4, 'train4.pkl')    # 训练集
joblib.dump(test02, 'test02.pkl')    # 训练集

['test02.pkl']